# Basics

Let's first take a look at what's inside the ``ib_insync`` package:

In [1]:
import ib_insync 
import ibapi

print(ib_insync.__all__)
print('what')

['util', 'Object', 'ContractDetails', 'ContractDescription', 'ComboLeg', 'UnderComp', 'DeltaNeutralContract', 'OrderComboLeg', 'OrderState', 'ScannerSubscription', 'SoftDollarTier', 'PriceIncrement', 'Execution', 'CommissionReport', 'ExecutionFilter', 'BarList', 'BarDataList', 'RealTimeBarList', 'BarData', 'RealTimeBar', 'HistogramData', 'TickAttrib', 'NewsProvider', 'DepthMktDataDescription', 'PnL', 'PnLSingle', 'AccountValue', 'RealTimeBar', 'TickData', 'TickByTickAllLast', 'TickByTickBidAsk', 'TickByTickMidPoint', 'HistoricalTick', 'HistoricalTickBidAsk', 'HistoricalTickLast', 'MktDepthData', 'DOMLevel', 'BracketOrder', 'TradeLogEntry', 'ScanData', 'TagValue', 'PortfolioItem', 'Position', 'Fill', 'OptionComputation', 'OptionChain', 'Dividends', 'NewsArticle', 'HistoricalNews', 'NewsTick', 'NewsBulletin', 'ConnectionStats', 'OrderCondition', 'ExecutionCondition', 'OperatorCondition', 'MarginCondition', 'ContractCondition', 'TimeCondition', 'PriceCondition', 'PercentChangeCondition', 

### Importing
The following two lines are used at the top of all notebooks. The first line imports everything and the second
starts an event loop to keep the notebook live updated:

In [2]:
from ib_insync import *
util.startLoop()
print(util.startLoop())


None


*Note that startLoop() only works in notebooks, not in regular Python programs.*

### Connecting
The main player of the whole package is the "IB" class. Let's create an IB instance and connect to a running TWS/IBG application:

In [3]:
ib = IB()
ib.connect('', 7497, clientId=14)

<IB connected to :7497 clientId=14>

If the connection failed, then verify that the application has the API port enabled and double-check the hostname and port. For IB Gateway the default port is 4002. Make sure the clientId is not already in use.

If the connection succeeded, then ib will be synchronized with TWS/IBG. The "current state" is now available via methods such as ib.positions(), ib.trades(), ib.openTrades(), ib.accountValues() or ib.tickers(). Let's list the current positions:

In [4]:
ib.positions()

[Position(account='DU1113767', contract=Contract(secType='STK', conId=3691937, symbol='AMZN', exchange='NASDAQ', currency='USD', localSymbol='AMZN', tradingClass='NMS'), position=100.0, avgCost=1845.91),
 Position(account='DU1113767', contract=Contract(secType='STK', conId=208813720, symbol='GOOG', exchange='NASDAQ', currency='USD', localSymbol='GOOG', tradingClass='NMS'), position=210.0, avgCost=1200.695),
 Position(account='DU1113767', contract=Contract(secType='FUT', conId=310629209, symbol='ES', lastTradeDateOrContractMonth='20190621', multiplier='50', currency='USD', localSymbol='ESM9', tradingClass='ES'), position=1.0, avgCost=141777.05)]

Or filter the account values to get the liquidation value:

In [14]:
[v for v in ib.accountValues() if v.tag == 'NetLiquidationByCurrency' and v.currency == 'BASE']

[AccountValue(account='DU1113767', tag='NetLiquidationByCurrency', value='1004256', currency='BASE', modelCode='')]

The "current state" will automatically be kept in sync with TWS/IBG. So an order fill will be added as soon as it is reported, or account values will be updated as soon as they change in TWS.

### Contracts

Contracts can be specified in different ways:
* The ibapi way, by creating an empty Contract object and setting its attributes one by one;
* By using Contract and giving the attributes as keyword argument;
* By using the specialized Stock, Option, Future, Forex, Index, CFD, Commodity,
  Bond, FuturesOption, MutualFund or Warrant contracts.

Some examples:

In [15]:
Contract(conId=270639)
Stock('AMD', 'SMART', 'USD')
Stock('INTC', 'SMART', 'USD', primaryExchange='NASDAQ')
Forex('EURUSD')
CFD('IBUS30')
Future('ES', '20180921', 'GLOBEX')
Option('SPY', '20170721', 240, 'C', 'SMART')
Bond(secIdType='ISIN', secId='US03076KAA60');

### Sending a request

The IB class has nearly all request methods that the IB API offers. The methods that return a result will block until finished and then return the result. Take for example reqContractDetails:

In [16]:
contract = Stock('AMZN', 'SMART', 'USD')
ib.reqContractDetails(contract)

[ContractDetails(contract=Contract(secType='STK', conId=3691937, symbol='AMZN', exchange='SMART', primaryExchange='NASDAQ', currency='USD', localSymbol='AMZN', tradingClass='NMS'), marketName='NMS', minTick=0.01, orderTypes='ACTIVETIM,ADJUST,ALERT,ALGO,ALLOC,AON,AVGCOST,BASKET,COND,CONDORDER,DARKONLY,DARKPOLL,DAY,DEACT,DEACTDIS,DEACTEOD,DIS,GAT,GTC,GTD,GTT,HID,IBKRATS,ICE,IMB,IOC,LIT,LMT,LOC,MIT,MKT,MOC,MTL,NGCOMB,NODARK,NONALGO,OCA,OPG,OPGREROUT,PEGBENCH,POSTONLY,PREOPGRTH,REL,RPI,RTH,SCALE,SCALEODD,SCALERST,SMARTSTG,SNAPMID,SNAPMKT,SNAPREL,STP,STPLMT,SWEEP,TRAIL,TRAILLIT,TRAILLMT,TRAILMIT,WHATIF', validExchanges='SMART,AMEX,NYSE,CBOE,ISE,CHX,ARCA,ISLAND,VWAP,DRCTEDGE,BEX,BATS,EDGEA,CSFBALGO,JEFFALGO,BYX,IEX,CVGXALGO,FOXRIVER,TPLUS1,NYSENAT,PSX', priceMagnifier=1, longName='AMAZON.COM INC', industry='Communications', category='Internet', subcategory='E-Commerce/Products', timeZoneId='EST5EDT', tradingHours='20180805:CLOSED;20180806:0400-2000', liquidHours='20180805:CLOSED;20180806:093

### Current state vs request

Doing a request involves network traffic going up and down and can take considerable time. The current state on the other hand is always immediately available. So it is preferable to use the current state methods over requests. For example, use ``ib.openOrders()`` in preference over ``ib.reqOpenOrders()``, or ``ib.positions()`` over ``ib.reqPositions()``, etc:

In [17]:
%time l = ib.positions()

CPU times: user 11 µs, sys: 1 µs, total: 12 µs
Wall time: 16.2 µs


In [35]:
%time l = ib.reqPositions()

CPU times: user 1.82 ms, sys: 1.53 ms, total: 3.35 ms
Wall time: 2.1 ms


### Logging

The following will put log messages of INFO and higher level under the current active cell:

In [18]:
util.logToConsole()

To see all debug messages (including network traffic):

In [19]:
import logging
util.logToConsole(logging.DEBUG)

### Disconnecting

The following will disconnect ``ib`` and clear all its state:

In [20]:
ib.disconnect()

2018-08-05 16:06:32,632 ib_insync.ib INFO Disconnecting from :7497, 146 B sent in 7 messages, 15.5 kB received in 314 messages, session time 36.8 s.
2018-08-05 16:06:32,639 ib_insync.client INFO Disconnected
2018-08-05 16:07:06,869 ib_insync.client DEBUG <<< 6,2,Cushion,0.883625,,DU1113767
2018-08-05 16:07:06,873 ib_insync.client DEBUG <<< 8,1,16:07
2018-08-05 16:07:06,874 ib_insync.client DEBUG <<< 73,1,2,DU1113767,,Cushion,0.883625,
2018-08-05 16:07:06,875 ib_insync.client DEBUG <<< 8,1,16:07
2018-08-05 16:07:06,877 ib_insync.client DEBUG <<< 8,1,16:07
2018-08-05 16:07:06,878 ib_insync.client DEBUG <<< 73,1,2,DU1113767,,Cushion,0.883625,
2018-08-05 16:07:06,879 ib_insync.client DEBUG <<< 8,1,16:07
2018-08-05 16:07:06,880 ib_insync.client DEBUG <<< 73,1,2,DU1113767,,Cushion,0.883625,
2018-08-05 16:07:06,883 ib_insync.client DEBUG <<< 8,1,16:07
2018-08-05 16:07:06,884 ib_insync.client DEBUG <<< 73,1,2,DU1113767,,Cushion,0.883625,
2018-08-05 16:07:06,949 ib_insync.client DEBUG <<< 6,2,E

2018-08-05 16:07:07,057 ib_insync.client DEBUG <<< 8,1,16:07
2018-08-05 16:07:07,058 ib_insync.client DEBUG <<< 73,1,2,DU1113767,,FullExcessLiquidity,887381.66,USD
2018-08-05 16:07:07,059 ib_insync.client DEBUG <<< 8,1,16:07
2018-08-05 16:07:07,060 ib_insync.client DEBUG <<< 73,1,2,DU1113767,,EquityWithLoanValue-S,994468.17,USD
2018-08-05 16:07:07,061 ib_insync.client DEBUG <<< 73,1,2,DU1113767,,EquityWithLoanValue-C,2661.66,USD
2018-08-05 16:07:07,064 ib_insync.client DEBUG <<< 73,1,2,DU1113767,,NetLiquidation-C,9767.86,USD
2018-08-05 16:07:07,065 ib_insync.client DEBUG <<< 73,1,2,DU1113767,,TotalCashValue-C,9767.86,USD
2018-08-05 16:07:07,069 ib_insync.client DEBUG <<< 73,1,2,DU1113767,,EquityWithLoanValue,997129.83,USD
2018-08-05 16:07:07,071 ib_insync.client DEBUG <<< 73,1,2,DU1113767,,NetLiquidation,1004251.11,USD
2018-08-05 16:07:07,072 ib_insync.client DEBUG <<< 73,1,2,DU1113767,,NetLiquidationUncertainty,15.08,USD
2018-08-05 16:07:07,074 ib_insync.client DEBUG <<< 73,1,2,DU1113

2018-08-05 16:07:07,219 ib_insync.client DEBUG <<< 6,2,UnrealizedPnL,3562,BASE,DU1113767
2018-08-05 16:07:07,220 ib_insync.client DEBUG <<< 8,1,16:07
2018-08-05 16:07:07,221 ib_insync.client DEBUG <<< 73,1,2,DU1113767,,Currency,BASE,BASE
2018-08-05 16:07:07,222 ib_insync.client DEBUG <<< 73,1,2,DU1113767,,CashBalance,565198,BASE
2018-08-05 16:07:07,224 ib_insync.client DEBUG <<< 73,1,2,DU1113767,,TotalCashBalance,565198,BASE
2018-08-05 16:07:07,225 ib_insync.client DEBUG <<< 73,1,2,DU1113767,,AccruedCash,0,BASE
2018-08-05 16:07:07,228 ib_insync.client DEBUG <<< 73,1,2,DU1113767,,StockMarketValue,439053,BASE
2018-08-05 16:07:07,230 ib_insync.client DEBUG <<< 73,1,2,DU1113767,,OptionMarketValue,0,BASE
2018-08-05 16:07:07,231 ib_insync.client DEBUG <<< 73,1,2,DU1113767,,FutureOptionValue,0,BASE
2018-08-05 16:07:07,232 ib_insync.client DEBUG <<< 73,1,2,DU1113767,,FuturesPNL,83,BASE
2018-08-05 16:07:07,234 ib_insync.client DEBUG <<< 73,1,2,DU1113767,,NetLiquidationByCurrency,1004251,BASE
20

2018-08-05 16:10:07,048 ib_insync.client DEBUG <<< 73,1,2,DU1113767,,EquityWithLoanValue,997149.91,USD
2018-08-05 16:10:07,049 ib_insync.client DEBUG <<< 6,2,NetLiquidation,1004256.11,USD,DU1113767
2018-08-05 16:10:07,050 ib_insync.client DEBUG <<< 73,1,2,DU1113767,,NetLiquidation,1004256.11,USD
2018-08-05 16:10:07,051 ib_insync.client DEBUG <<< 6,2,TotalCashValue,565203.12,USD,DU1113767
2018-08-05 16:10:07,052 ib_insync.client DEBUG <<< 73,1,2,DU1113767,,TotalCashValue,565203.12,USD
2018-08-05 16:10:07,053 ib_insync.client DEBUG <<< 6,2,BuyingPower,3549546.65,USD,DU1113767
2018-08-05 16:10:07,054 ib_insync.client DEBUG <<< 73,1,2,DU1113767,,BuyingPower,3549546.65,USD
2018-08-05 16:10:07,055 ib_insync.client DEBUG <<< 6,2,AvailableFunds-S,884720.00,USD,DU1113767
2018-08-05 16:10:07,056 ib_insync.client DEBUG <<< 73,1,2,DU1113767,,AvailableFunds-S,884720.00,USD
2018-08-05 16:10:07,057 ib_insync.client DEBUG <<< 6,2,AvailableFunds-C,-7992.63,USD,DU1113767
2018-08-05 16:10:07,058 ib_insyn

2018-08-05 16:10:07,129 ib_insync.client DEBUG <<< 73,1,2,DU1113767,,LookAheadExcessLiquidity,887386.66,USD
2018-08-05 16:10:07,130 ib_insync.client DEBUG <<< 73,1,2,DU1113767,,FullAvailableFunds-S,884720.00,USD
2018-08-05 16:10:07,131 ib_insync.client DEBUG <<< 73,1,2,DU1113767,,FullAvailableFunds-C,-7992.63,USD
2018-08-05 16:10:07,133 ib_insync.client DEBUG <<< 73,1,2,DU1113767,,FullExcessLiquidity-C,2666.67,USD
2018-08-05 16:10:07,134 ib_insync.client DEBUG <<< 73,1,2,DU1113767,,FullAvailableFunds,876727.36,USD
2018-08-05 16:10:07,135 ib_insync.client DEBUG <<< 73,1,2,DU1113767,,FullExcessLiquidity,887386.66,USD
2018-08-05 16:10:07,136 ib_insync.client DEBUG <<< 8,1,16:10
2018-08-05 16:10:07,136 ib_insync.client DEBUG <<< 73,1,2,DU1113767,,EquityWithLoanValue-S,994483.25,USD
2018-08-05 16:10:07,138 ib_insync.client DEBUG <<< 73,1,2,DU1113767,,NetLiquidationUncertainty,0.00,USD
2018-08-05 16:10:07,139 ib_insync.client DEBUG <<< 73,1,2,DU1113767,,EquityWithLoanValue-C,2666.67,USD
2018

...

2018-08-05 16:49:35,673 ib_insync.wrapper INFO position: Position(account='DU1113767', contract=Contract(secType='STK', conId=208813720, symbol='GOOG', exchange='NASDAQ', currency='USD', localSymbol='GOOG', tradingClass='NMS'), position=210.0, avgCost=1200.695)
2018-08-05 16:49:35,676 ib_insync.client DEBUG <<< 61,3,DU1113767,310629209,ES,FUT,20190621,0.0,,50,,USD,ESM9,ES,1,141777.05
2018-08-05 16:49:35,682 ib_insync.wrapper INFO position: Position(account='DU1113767', contract=Contract(secType='FUT', conId=310629209, symbol='ES', lastTradeDateOrContractMonth='20190621', multiplier='50', currency='USD', localSymbol='ESM9', tradingClass='ES'), position=1.0, avgCost=141777.05)
2018-08-05 16:49:35,739 ib_insync.client DEBUG <<< 4,2,-1,1102,Connectivity between IB and Trader Workstation has been restored - data maintained.
2018-08-05 16:49:35,743 ib_insync.wrapper ERROR Error 1102, reqId -1: Connectivity between IB and Trader Workstation has been restored - data maintained.
2018-08-05 16:4

2018-08-05 16:49:36,388 ib_insync.client DEBUG <<< 6,2,ExcessLiquidity-C,2617.25,USD,DU1113767
2018-08-05 16:49:36,393 ib_insync.client DEBUG <<< 73,1,2,DU1113767,,ExcessLiquidity-C,2617.25,USD
2018-08-05 16:49:36,398 ib_insync.client DEBUG <<< 73,1,2,DU1113767,,ExcessLiquidity-S,884720.00,USD
2018-08-05 16:49:36,408 ib_insync.client DEBUG <<< 6,2,FullAvailableFunds,876438.51,USD,DU1113767
2018-08-05 16:49:36,415 ib_insync.client DEBUG <<< 73,1,2,DU1113767,,FullAvailableFunds,876438.51,USD
2018-08-05 16:49:36,422 ib_insync.client DEBUG <<< 6,2,FullAvailableFunds-C,-8042.05,USD,DU1113767
2018-08-05 16:49:36,432 ib_insync.client DEBUG <<< 73,1,2,DU1113767,,FullAvailableFunds-C,-8042.05,USD
2018-08-05 16:49:36,441 ib_insync.client DEBUG <<< 6,2,FullAvailableFunds-S,884480.56,USD,DU1113767
2018-08-05 16:49:36,447 ib_insync.client DEBUG <<< 73,1,2,DU1113767,,FullAvailableFunds-S,884480.56,USD
2018-08-05 16:49:36,456 ib_insync.client DEBUG <<< 6,2,FullExcessLiquidity,887337.25,USD,DU1113767


2018-08-05 16:49:36,889 ib_insync.client DEBUG <<< 73,1,2,DU1113767,,AccruedCash-C,0.00,USD
2018-08-05 16:49:36,894 ib_insync.client DEBUG <<< 73,1,2,DU1113767,,AccruedCash-S,0.00,USD
2018-08-05 16:49:36,898 ib_insync.client DEBUG <<< 73,1,2,DU1113767,,AccruedDividend,0.00,USD
2018-08-05 16:49:36,904 ib_insync.client DEBUG <<< 73,1,2,DU1113767,,AccruedDividend-C,0.00,USD
2018-08-05 16:49:36,908 ib_insync.client DEBUG <<< 73,1,2,DU1113767,,AccruedDividend-S,0.00,USD
2018-08-05 16:49:36,917 ib_insync.client DEBUG <<< 73,1,2,DU1113767,,AvailableFunds,876438.51,USD
2018-08-05 16:49:36,922 ib_insync.client DEBUG <<< 73,1,2,DU1113767,,AvailableFunds-C,-8042.05,USD
2018-08-05 16:49:36,926 ib_insync.client DEBUG <<< 73,1,2,DU1113767,,AvailableFunds-S,884480.56,USD
2018-08-05 16:49:36,932 ib_insync.client DEBUG <<< 73,1,2,DU1113767,,Billable,0.00,USD
2018-08-05 16:49:36,939 ib_insync.client DEBUG <<< 73,1,2,DU1113767,,Billable-C,0.00,USD
2018-08-05 16:49:36,944 ib_insync.client DEBUG <<< 73,1,2

2018-08-05 16:49:37,313 ib_insync.client DEBUG <<< 73,1,2,DU1113767,,AccruedCash-C,0.00,USD
2018-08-05 16:49:37,317 ib_insync.client DEBUG <<< 73,1,2,DU1113767,,AccruedCash-S,0.00,USD
2018-08-05 16:49:37,321 ib_insync.client DEBUG <<< 73,1,2,DU1113767,,AccruedDividend,0.00,USD
2018-08-05 16:49:37,325 ib_insync.client DEBUG <<< 73,1,2,DU1113767,,AccruedDividend-C,0.00,USD
2018-08-05 16:49:37,331 ib_insync.client DEBUG <<< 73,1,2,DU1113767,,AccruedDividend-S,0.00,USD
2018-08-05 16:49:37,335 ib_insync.client DEBUG <<< 73,1,2,DU1113767,,AvailableFunds,876438.51,USD
2018-08-05 16:49:37,340 ib_insync.client DEBUG <<< 73,1,2,DU1113767,,AvailableFunds-C,-8042.05,USD
2018-08-05 16:49:37,346 ib_insync.client DEBUG <<< 73,1,2,DU1113767,,AvailableFunds-S,884480.56,USD
2018-08-05 16:49:37,352 ib_insync.client DEBUG <<< 73,1,2,DU1113767,,Billable,0.00,USD
2018-08-05 16:49:37,355 ib_insync.client DEBUG <<< 73,1,2,DU1113767,,Billable-C,0.00,USD
2018-08-05 16:49:37,360 ib_insync.client DEBUG <<< 73,1,2

2018-08-05 16:49:38,158 ib_insync.client DEBUG <<< 73,1,2,DU1113767,,ExchangeRate,1.00,BASE
2018-08-05 16:49:38,162 ib_insync.client DEBUG <<< 73,1,2,DU1113767,,FundValue,0,BASE
2018-08-05 16:49:38,167 ib_insync.client DEBUG <<< 73,1,2,DU1113767,,NetDividend,0,BASE
2018-08-05 16:49:38,171 ib_insync.client DEBUG <<< 73,1,2,DU1113767,,MutualFundValue,0,BASE
2018-08-05 16:49:38,176 ib_insync.client DEBUG <<< 73,1,2,DU1113767,,MoneyMarketFundValue,0,BASE
2018-08-05 16:49:38,181 ib_insync.client DEBUG <<< 73,1,2,DU1113767,,CorporateBondValue,0,BASE
2018-08-05 16:49:38,186 ib_insync.client DEBUG <<< 73,1,2,DU1113767,,TBondValue,0,BASE
2018-08-05 16:49:38,194 ib_insync.client DEBUG <<< 73,1,2,DU1113767,,TBillValue,0,BASE
2018-08-05 16:49:38,199 ib_insync.client DEBUG <<< 73,1,2,DU1113767,,WarrantValue,0,BASE
2018-08-05 16:49:38,206 ib_insync.client DEBUG <<< 73,1,2,DU1113767,,FxCashBalance,0,BASE
2018-08-05 16:49:38,212 ib_insync.client DEBUG <<< 73,1,2,DU1113767,,AccountOrGroup,DU1113767,BAS